# ベーシックステップ第1回課題
銀行のキャンペーン

In [2]:
#ライブラリインポート
%matplotlib inline
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import scipy as sp
from scipy import stats

import psycopg2
from pandas import DataFrame
from __future__ import print_function

#Scikit-learnをインポート
from sklearn.ensemble import RandomForestClassifier
#from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
#from sklearn.metrics import accuracy_score
#from sklearn.metrics import mean_squared_error

# データのインポート

In [52]:
#学習データのインポート
train_data = pd.read_csv('data/bank_marketing_train.csv')

In [53]:
#学習データを分割
train_data1 = train_data[train_data['poutcome']!="success"]
train_data2 = train_data[train_data['poutcome']=="success"]

In [54]:
#実際のデータのインポート
test_data = pd.read_csv('data/bank_marketing_test.csv')

In [55]:
#実際のデータを分割
test_data1 = test_data[test_data['poutcome']!="success"]
test_data2 = test_data[test_data['poutcome']=="success"]

# データセット1の処理

In [56]:
#【データセット1】
#カテゴリ変数の展開
train_data1 = pd.get_dummies(data=train_data1, columns=['job'])
train_data1 = pd.get_dummies(data=train_data1, columns=['month'])
train_data1 = pd.get_dummies(data=train_data1, columns=['marital'])
train_data1 = pd.get_dummies(data=train_data1, columns=['education'])
train_data1 = pd.get_dummies(data=train_data1, columns=['default'])
train_data1 = pd.get_dummies(data=train_data1, columns=['housing'])
train_data1 = pd.get_dummies(data=train_data1, columns=['loan'])
train_data1 = pd.get_dummies(data=train_data1, columns=['contact'])

In [57]:
#ターゲット変数と説明変数の設定
target_col = 'y'
exclude_cols = ['day_of_week','duration','poutcome']
feature_cols = [col for col in train_data1.columns if col not in exclude_cols]

#説明変数のデータ
y = train_data1[target_col]
x = train_data1[feature_cols]

In [58]:
#学習データを、学習用とパラメータチューニング用に分割
feature2_cols = [col for col in x.columns if col not in target_col]
x = x[feature2_cols]

X_train1, X_train2, Y_train1, Y_train2 = \
    train_test_split(x, y, test_size=0.3, random_state=1234)

In [59]:
#ランダムフォレスト
clf1 = RandomForestClassifier(n_estimators = 400, max_depth = 50, class_weight = "balanced", max_leaf_nodes = 1000)
clf1.fit(X_train1, Y_train1)
clf1.score(X_train1, Y_train1)

0.96989849492474622

In [60]:
#モデルに伴う生存率の予測値
Y_pred2 = clf1.predict(X_train2)

In [68]:
# Feature Importance
fti = clf1.feature_importances_


[  1.67474992e-01   9.85156538e-02   2.30783656e-04   1.42185137e-02
   2.73564705e-02   3.64603950e-02   1.85650669e-02   1.51091324e-01
   2.48091622e-02   2.02269388e-02   1.89696133e-02   9.33432567e-03
   5.95623142e-03   1.15831545e-02   7.78989277e-03   8.84879428e-03
   1.48088628e-02   3.91230393e-03   1.65195737e-02   5.25955136e-03
   2.21909855e-03   3.41709770e-03   2.76915162e-03   5.02193419e-05
   3.24729643e-03   4.32160022e-03   3.14070626e-03   1.02864389e-02
   1.74664503e-03   6.49220588e-03   1.45395844e-02   2.04148524e-02
   1.81954693e-02   1.47376253e-03   1.28007254e-02   1.06414399e-02
   1.62125757e-02   1.98757412e-02   2.50134889e-04   1.42175188e-02
   1.95304770e-02   7.68071762e-03   1.61978898e-02   1.62332521e-02
   1.31453565e-06   2.23277764e-02   3.94951364e-03   2.21225452e-02
   1.73417303e-02   3.81284501e-03   1.67876072e-02   1.21505471e-02
   1.36199150e-02]


# 実際の予測とファイルの作成

In [61]:
test_data1_preserve = test_data1
#test_data1_preserve.head()

In [62]:
#【データセット1】
#カテゴリ変数の展開
test_data1 = pd.get_dummies(data=test_data1, columns=['job'])
test_data1 = pd.get_dummies(data=test_data1, columns=['month'])
test_data1 = pd.get_dummies(data=test_data1, columns=['marital'])
test_data1 = pd.get_dummies(data=test_data1, columns=['education'])
test_data1 = pd.get_dummies(data=test_data1, columns=['default'])
test_data1 = pd.get_dummies(data=test_data1, columns=['housing'])
test_data1 = pd.get_dummies(data=test_data1, columns=['loan'])
test_data1 = pd.get_dummies(data=test_data1, columns=['contact'])

In [63]:
#テストデータ1から予測
#ターゲット変数と説明変数の設定
exclude_cols = ['day_of_week','duration','poutcome','y']
feature_cols = [col for col in test_data1.columns if col not in exclude_cols]

#説明変数のデータ
X_test1 = test_data1[feature_cols]
Y_pred = clf1.predict(X_test1)

In [64]:
#output = DataFrame({'age': X_test1['age'], 'y': Y_pred})
output = DataFrame({'age': test_data1_preserve['age'], 'job': test_data1_preserve['job'], 'marital': test_data1_preserve['marital'], 'education': test_data1_preserve['education'], 'default': test_data1_preserve['default'], 'housing': test_data1_preserve['housing'], 'loan': test_data1_preserve['loan'], 'contact': test_data1_preserve['contact'], 'month': test_data1_preserve['month'], 'day_of_week': test_data1_preserve['day_of_week'], 'duration': test_data1_preserve['duration'], 'campaign': test_data1_preserve['campaign'], 'pdays': test_data1_preserve['pdays'], 'previous': test_data1_preserve['previous'], 'poutcome': test_data1_preserve['poutcome'], 'emp.var.rate': test_data1_preserve['emp.var.rate'], 'cons.price.idx': test_data1_preserve['cons.price.idx'], 'cons.conf.idx': test_data1_preserve['cons.conf.idx'], 'euribor3m': test_data1_preserve['euribor3m'], 'nr.employed': test_data1_preserve['nr.employed'],  'y': test_data1_preserve['y'],  'y_pred': Y_pred})

In [65]:
#テストデータ2にすべてYesを設定
test_data2['Y_predict'] = "yes"
test_data2.head()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,Y_predict
2,40,technician,married,high.school,no,yes,no,cellular,jul,wed,...,3,1,success,-2.9,92.469,-33.6,1.029,5076.2,no,yes
35,54,admin.,married,university.degree,no,no,yes,cellular,jul,tue,...,13,1,success,-2.9,92.469,-33.6,0.908,5076.2,yes,yes
53,42,admin.,married,university.degree,no,yes,no,cellular,aug,tue,...,4,1,success,-2.9,92.201,-31.4,0.884,5076.2,yes,yes
56,31,admin.,single,university.degree,no,no,no,cellular,aug,tue,...,15,1,success,-2.9,92.201,-31.4,0.884,5076.2,no,yes
60,29,student,single,high.school,no,unknown,unknown,cellular,aug,tue,...,4,3,success,-2.9,92.201,-31.4,0.884,5076.2,yes,yes


In [66]:
output2 = DataFrame({'age': test_data2['age'], 'job': test_data2['job'], 'marital': test_data2['marital'], 'education': test_data2['education'], 'default': test_data2['default'], 'housing': test_data2['housing'], 'loan': test_data2['loan'], 'contact': test_data2['contact'], 'month': test_data2['month'], 'day_of_week': test_data2['day_of_week'], 'duration': test_data2['duration'], 'campaign': test_data2['campaign'], 'pdays': test_data2['pdays'], 'previous': test_data2['previous'], 'poutcome': test_data2['poutcome'], 'emp.var.rate': test_data2['emp.var.rate'], 'cons.price.idx': test_data2['cons.price.idx'], 'cons.conf.idx': test_data2['cons.conf.idx'], 'euribor3m': test_data2['euribor3m'], 'nr.employed': test_data2['nr.employed'],  'y': test_data2['y'], 'y_pred': test_data2['Y_predict']})
output = output.append(output2)
output.head()

,age,campaign,cons.conf.idx,cons.price.idx,contact,day_of_week,default,duration,education,emp.var.rate,...,job,loan,marital,month,nr.employed,pdays,poutcome,previous,y,y_pred
0,32,5,-33.6,92.469,cellular,wed,no,210,university.degree,-2.9,...,admin.,no,single,jul,5076.2,999,nonexistent,0,yes,no
1,59,2,-33.6,92.469,cellular,wed,no,286,high.school,-2.9,...,admin.,no,married,jul,5076.2,999,nonexistent,0,no,no
3,26,1,-33.6,92.469,cellular,wed,no,153,university.degree,-2.9,...,admin.,no,single,jul,5076.2,999,nonexistent,0,yes,no
4,36,3,-33.6,92.469,cellular,wed,no,182,professional.course,-2.9,...,technician,no,married,jul,5076.2,999,nonexistent,0,no,no
5,32,3,-33.6,92.469,cellular,wed,no,97,university.degree,-2.9,...,admin.,no,single,jul,5076.2,999,nonexistent,0,yes,no


In [67]:
#データの書き出し
outfile = DataFrame({'age': output['age'], 'job': output['job'], 'marital': output['marital'], 'education': output['education'], 'default': output['default'], 'housing': output['housing'], 'loan': output['loan'], 'contact': output['contact'], 'month': output['month'], 'day_of_week': output['day_of_week'], 'duration': output['duration'], 'campaign': output['campaign'], 'pdays': output['pdays'], 'previous': output['previous'], 'poutcome': output['poutcome'], 'emp.var.rate': output['emp.var.rate'], 'cons.price.idx': output['cons.price.idx'], 'cons.conf.idx': output['cons.conf.idx'], 'euribor3m': output['euribor3m'], 'nr.employed': output['nr.employed'],  'y': output['y'],  'y_pred': output['y_pred']})
outfile.to_csv("output.csv", index=False)